In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['./cnn_finetune'])

from load_cifar import load_cifar10_data, load_cifar100_data
from keras.utils import multi_gpu_model
import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

tensorflow 1.13.1
keras 2.2.4


In [3]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from alt_model_checkpoint import AltModelCheckpoint

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [4]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from cnn_finetune.resnet_152 import resnet152_model
from cnn_finetune.load_cifar import load_cifar10_data, load_cifar100_data

In [5]:
# Example to fine-tune on 3000 samples from Cifar10
gpus = 2
model_name = 'resnet_152_GPU23_e03_2.9979_val_val'
dataset_name = 'imaterialist'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 2019
batch_size = 16*gpus
print(">>> batch_size:", batch_size)
epochs = 100
load_weights = False
fine_tuning = False

# Load our model
model = resnet152_model(img_rows, img_cols, channel, num_classes, fine_tuning, load_weights)

>>> batch_size: 32
Instructions for updating:
Colocations handled automatically by placer.
load_weights False
resnet152_model: classes: 2019 fine-tuning: True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 

__________________________________________________________________________________________________
res4b20_branch2a_relu (Activati (None, 14, 14, 256)  0           scale4b20_branch2a[0][0]         
__________________________________________________________________________________________________
res4b20_branch2b_zeropadding (Z (None, 16, 16, 256)  0           res4b20_branch2a_relu[0][0]      
__________________________________________________________________________________________________
res4b20_branch2b (Conv2D)       (None, 14, 14, 256)  589824      res4b20_branch2b_zeropadding[0][0
__________________________________________________________________________________________________
bn4b20_branch2b (BatchNormaliza (None, 14, 14, 256)  1024        res4b20_branch2b[0][0]           
__________________________________________________________________________________________________
scale4b20_branch2b (Scale)      (None, 14, 14, 256)  512         bn4b20_branch2b[0][0]            
__________

In [6]:
weights_path = '/tf/imaterialist-product-2019/models/resnet_152/imaterialist-e03-2.9979.hd5'
model.load_weights(weights_path, by_name=True)

In [7]:
from lib.DataGenerator import DataGenerator
# train_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size)
train_data_generator = DataGenerator(file_type='train', corp=True, batch_size=batch_size)
val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size)

Found 1008641 images belonging to 2019 classes.
Found 10095 images belonging to 2019 classes.


In [8]:
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)
# steps_per_epoch = 10
# validation_steps = len(val_data_generator)


In [9]:
print(steps_per_epoch)
print(validation_steps)

31521
316


In [10]:
# Learning rate is changed to 0.001
# adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

parallel_model = multi_gpu_model(model, gpus=gpus)
parallel_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

parallel_model.summary()
checkpoint = AltModelCheckpoint(filepath="./models/" + model_name + "/"+ dataset_name +"-e" + "{epoch:02d}-{val_loss:.4f}.hd5", 
                             alternate_model = model,
                                monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=True, 
                             mode='auto', 
                             period=1)

checkpoint_best = AltModelCheckpoint(filepath="./models/" + model_name + "/"+ dataset_name + "-best.hd5", 
                                  alternate_model = model,
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=True,
                                  mode='auto', 
                                  period=1)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

tensorBoard = TensorBoard(log_dir='./models/' + model_name +'/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

csvLogger = CSVLogger(filename='./models/'+ model_name + '/' + dataset_name + "-log.csv", separator=',', append=False)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
model_2 (Model)                 (None, 2019)         62583587    lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
__________

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [11]:
valid_evaluate = parallel_model.evaluate_generator(val_data_generator, 
                                     steps = len(val_data_generator),
                                     workers=50, use_multiprocessing=True,
                                     verbose=1)
print(valid_evaluate)

316/316 [==============================] - 120s 380ms/step
[2.9978917765463855, 0.35740465579084857]


In [12]:
parallel_model.fit_generator(val_data_generator,
          steps_per_epoch = validation_steps,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=val_data_generator,
          validation_steps=validation_steps,
          workers=50,
          use_multiprocessing=True,
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
316/316 [==============================] - 413s 1s/step - loss: 2.5487 - acc: 0.4107 - val_loss: 1.7965 - val_acc: 0.5636

Epoch 00001: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e01-1.7965.hd5

Epoch 00001: val_loss improved from inf to 1.79650, saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-best.hd5
Epoch 2/100
316/316 [==============================] - 307s 972ms/step - loss: 1.5861 - acc: 0.6002 - val_loss: 1.3685 - val_acc: 0.6606

Epoch 00002: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e02-1.3685.hd5

Epoch 00002: val_loss improved from 1.79650 to 1.36855, saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-best.hd5
Epoch 3/100
316/316 [==============================] - 287s 910ms/step - loss: 0.9004 - acc: 0.7720 - val_loss: 0.8163 - val_acc: 0.8038

Epoch 00003: saving model to ./models/resnet_152_GPU23_e03_2.


Epoch 00020: val_loss improved from 0.00712 to 0.00672, saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-best.hd5
Epoch 21/100
316/316 [==============================] - 286s 904ms/step - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0070 - val_acc: 0.9992

Epoch 00021: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e21-0.0070.hd5

Epoch 00021: val_loss did not improve from 0.00672
Epoch 22/100
315/316 [============================>.] - ETA: 0s - loss: 0.0048 - acc: 0.9997
Epoch 00021: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e21-0.0070.hd5
316/316 [==============================] - 287s 908ms/step - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0068 - val_acc: 0.9991

Epoch 00022: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e22-0.0068.hd5

Epoch 00022: val_loss did not improve from 0.00672
Epoch 23/100
316/316 [==============================] - 281s 890ms/step - loss: 0.0048 - acc

316/316 [==============================] - 284s 899ms/step - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0065 - val_acc: 0.9992

Epoch 00043: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e43-0.0065.hd5

Epoch 00043: val_loss did not improve from 0.00639
Epoch 44/100
316/316 [==============================] - 282s 894ms/step - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0064 - val_acc: 0.9994

Epoch 00044: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e44-0.0064.hd5

Epoch 00044: val_loss did not improve from 0.00639
Epoch 45/100
316/316 [==============================] - 285s 902ms/step - loss: 0.0048 - acc: 0.9997 - val_loss: 0.0064 - val_acc: 0.9993

Epoch 00045: saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-e45-0.0064.hd5

Epoch 00045: val_loss improved from 0.00639 to 0.00635, saving model to ./models/resnet_152_GPU23_e03_2.9979_val_val/imaterialist-best.hd5

Epoch 00045: ReduceLROnPlateau reducing learn

In [14]:
parallel_model.evaluate_generator(train_data_generator, 
                                     steps = len(train_data_generator),
                                     workers=50, use_multiprocessing=True,
                                     verbose=1)

31521/31521 [==============================] - 11049s 351ms/step


[7.772719023425045, 0.30607718702690057]

In [12]:
parallel_model.fit_generator(train_data_generator,
          steps_per_epoch = steps_per_epoch,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=val_data_generator,
          validation_steps=validation_steps,
          workers=50,
          use_multiprocessing=True,
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
31521/31521 [==============================] - 21167s 672ms/step - loss: 3.2952 - acc: 0.3982 - val_loss: 7.7043 - val_acc: 4.9529e-04

Epoch 00001: saving model to ./models/resnet_152_GPU23_e03_2.9979/imaterialist-e01-7.7043.hd5


Process ForkPoolWorker-249:
Process ForkPoolWorker-237:
Process ForkPoolWorker-201:
Process ForkPoolWorker-206:
Process ForkPoolWorker-218:
Process ForkPoolWorker-266:
Process ForkPoolWorker-244:
Process ForkPoolWorker-246:
Process ForkPoolWorker-216:
Process ForkPoolWorker-202:
Process ForkPoolWorker-287:
Process ForkPoolWorker-298:
Process ForkPoolWorker-219:
Process ForkPoolWorker-242:
Process ForkPoolWorker-209:
Process ForkPoolWorker-296:
Process ForkPoolWorker-259:
Process ForkPoolWorker-286:
Process ForkPoolWorker-272:
Process ForkPoolWorker-257:
Process ForkPoolWorker-215:
Process ForkPoolWorker-230:
Process ForkPoolWorker-282:
Process ForkPoolWorker-261:
Process ForkPoolWorker-294:
Process ForkPoolWorker-243:
Process ForkPoolWorker-275:
Process ForkPoolWorker-270:
Process ForkPoolWorker-262:
Process ForkPoolWorker-205:
Process ForkPoolWorker-273:
Process ForkPoolWorker-293:
Process ForkPoolWorker-263:
Process ForkPoolWorker-217:
Process ForkPoolWorker-220:
Process ForkPoolWork

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/li

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in 

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._t

  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/u

  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  Fi

KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__

KeyboardInterrupt: 

In [ ]:
# Make predictions
# predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
# print(Y_valid.shape, predictions_valid.shape)

In [ ]:
# import numpy as np
# np.set_printoptions(threshold=np.inf)
# print(Y_valid)
# print(predictions_valid)

In [ ]:
# print('cross-entropy:', sklearn.metrics.log_loss(Y_valid, predictions_valid))

# y_true = np.argmax(Y_valid, axis=1)
# y_hat = np.argmax(predictions_valid, axis=1)
# # print('report:', sklearn.metrics.classification_report(y_true, y_hat))
# print('acc:', sklearn.metrics.accuracy_score(y_true, y_hat))
# # print('roc:', sklearn.metrics.roc_auc_score(y_true, y_hat))
# # print('confusion_matrix:\n', sklearn.metrics.confusion_matrix(y_true, y_hat))